<a href="https://colab.research.google.com/github/bryantpwhite/ProjectRepo/blob/main/Code_for_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
# importing pandas
import pandas as pd
  
# read text file into pandas DataFrame
df = pd.read_fwf("Tristan.txt")


In [6]:
df[0]

KeyError: ignored

In [4]:
from google.colab import drive
drive.mount('/content/gdrive')
%cd gdrive/MyDrive/Colab\ Notebooks

Mounted at /content/gdrive
/content/gdrive/MyDrive/Colab Notebooks


In [7]:
! pip install transformers
! pip install datasets

     |████████████████████████████████| 3.4 MB 5.1 MB/s 
     |████████████████████████████████| 895 kB 64.5 MB/s 
     |████████████████████████████████| 3.3 MB 54.4 MB/s 
     |████████████████████████████████| 67 kB 7.8 MB/s 
     |████████████████████████████████| 596 kB 74.6 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 306 kB 5.1 MB/s 
     |████████████████████████████████| 1.1 MB 65.6 MB/s 
     |████████████████████████████████| 133 kB 77.2 MB/s 
     |████████████████████████████████| 243 kB 73.1 MB/s 
     |████████████████████████████████| 144 kB 74.2 MB/s 
     |████████████████████████████████| 160 kB 72.5 MB/s 
     |████████████████████████████████| 271 kB 75.1 MB/s 


In [8]:
import numpy as np
import pandas as pd

from datasets import load_dataset, load_metric, Dataset
from transformers import pipeline
from transformers import DataCollatorForLanguageModeling
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, CamembertTokenizer

from huggingface_hub import notebook_login


In [9]:
ds_path = 'bryantpwhite/Medieval_Texts_in_French'
demo_ds = load_dataset(ds_path, use_auth_token=True)

Downloading:   0%|          | 0.00/774 [00:00<?, ?B/s]

Using custom data configuration bryantpwhite--Medieval_Texts_in_French-213d8ecd3f6cc3b0


  0%|          | 0/2 [00:00<?, ?it/s]

Downloading:   0%|          | 0.00/156k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/218k [00:00<?, ?B/s]

  0%|          | 0/2 [00:00<?, ?it/s]

Dataset parquet downloaded and prepared to /root/.cache/huggingface/datasets/parquet/bryantpwhite--Medieval_Texts_in_French-213d8ecd3f6cc3b0/0.0.0/1638526fd0e8d960534e2155dc54fdff8dce73851f21f031d2fb9c2cf757c121. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [10]:
demo_ds

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 12140
    })
    test: Dataset({
        features: ['text'],
        num_rows: 18384
    })
})

In [11]:
from transformers import AutoModelForMaskedLM

model_checkpoint = "camembert-base"
model = AutoModelForMaskedLM.from_pretrained(model_checkpoint)

Downloading:   0%|          | 0.00/508 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/424M [00:00<?, ?B/s]

In [13]:
#instantiate tokenizer
tokenizer = AutoTokenizer.from_pretrained("camembert-base")

In [14]:
def tokenize_function(examples):
    result = tokenizer(examples["text"], truncation=True, padding=True)
    if tokenizer.is_fast:
        result["word_ids"] = [result.word_ids(i) for i in range(len(result["input_ids"]))]
    return result


# Use batched=True to activate fast multithreading!
tokenized_datasets = demo_ds.map(
    tokenize_function, batched=True, remove_columns=["text"]
)
tokenized_datasets

  0%|          | 0/13 [00:00<?, ?ba/s]

  0%|          | 0/19 [00:00<?, ?ba/s]

DatasetDict({
    train: Dataset({
        features: ['attention_mask', 'input_ids', 'word_ids'],
        num_rows: 12140
    })
    test: Dataset({
        features: ['attention_mask', 'input_ids', 'word_ids'],
        num_rows: 18384
    })
})

In [15]:
chunk_size = 128

In [16]:
# Slicing produces a list of lists for each feature
tokenized_samples = tokenized_datasets["train"][:3]

for idx, sample in enumerate(tokenized_samples["input_ids"]):
    print(f"'>>> Text {idx} length: {len(sample)}'")

'>>> Text 0 length: 29'
'>>> Text 1 length: 29'
'>>> Text 2 length: 29'


In [17]:
concatenated_examples = {
    k: sum(tokenized_samples[k], []) for k in tokenized_samples.keys()
}
total_length = len(concatenated_examples["input_ids"])
print(f"'>>> Concatenated text length: {total_length}'")

'>>> Concatenated text length: 87'


In [18]:
chunks = {
    k: [t[i : i + chunk_size] for i in range(0, total_length, chunk_size)]
    for k, t in concatenated_examples.items()
}

for chunk in chunks["input_ids"]:
    print(f"'>>> Chunk length: {len(chunk)}'")

'>>> Chunk length: 87'


In [19]:
def group_texts(examples):
    # Concatenate all texts
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    # Compute length of concatenated texts
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the last chunk if it's smaller than chunk_size
    total_length = (total_length // chunk_size) * chunk_size
    # Split by chunks of max_len
    result = {
        k: [t[i : i + chunk_size] for i in range(0, total_length, chunk_size)]
        for k, t in concatenated_examples.items()
    }
    # Create a new labels column
    result["labels"] = result["input_ids"].copy()
    return result

In [20]:
lm_datasets = tokenized_datasets.map(group_texts, batched=True)
lm_datasets

  0%|          | 0/13 [00:00<?, ?ba/s]

  0%|          | 0/19 [00:00<?, ?ba/s]

DatasetDict({
    train: Dataset({
        features: ['attention_mask', 'input_ids', 'labels', 'word_ids'],
        num_rows: 2491
    })
    test: Dataset({
        features: ['attention_mask', 'input_ids', 'labels', 'word_ids'],
        num_rows: 2999
    })
})

In [21]:
tokenizer.decode(lm_datasets["train"][1]["input_ids"])

'<pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><s></s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><s> Librairie ancienne Édouard Champion, Paris, 1929</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><s></s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><s></s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>'

In [22]:
tokenizer.decode(lm_datasets["train"][1]["labels"])

'<pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><s></s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><s> Librairie ancienne Édouard Champion, Paris, 1929</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><s></s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><s></s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>'

### Ashley's Troubleshooting Edits Start

In [23]:
demo_ds['train'][2]

{'text': ''}

In [24]:
for i in range(100):  
  if bool(demo_ds['train'][i]['text'])==False:
    print("Empty")
  else:
    print("Not Empty")

Not Empty
Empty
Empty
Empty
Empty
Empty
Not Empty
Empty
Empty
Empty
Empty
Empty
Not Empty
Empty
Empty
Empty
Empty
Empty
Not Empty
Empty
Empty
Empty
Empty
Empty
Not Empty
Empty
Empty
Empty
Empty
Empty
Not Empty
Empty
Not Empty
Empty
Not Empty
Empty
Not Empty
Empty
Not Empty
Empty
Not Empty
Empty
Not Empty
Empty
Not Empty
Empty
Not Empty
Empty
Not Empty
Empty
Not Empty
Empty
Not Empty
Empty
Not Empty
Empty
Not Empty
Empty
Not Empty
Empty
Empty
Empty
Empty
Empty
Not Empty
Empty
Empty
Not Empty
Empty
Not Empty
Empty
Not Empty
Empty
Not Empty
Empty
Not Empty
Empty
Not Empty
Empty
Empty
Empty
Not Empty
Empty
Empty
Empty
Not Empty
Empty
Not Empty
Empty
Not Empty
Empty
Not Empty
Empty
Not Empty
Empty
Not Empty
Empty
Not Empty
Empty
Not Empty


In [25]:
sample = demo_ds["train"].shuffle(seed=42).select(range(20))

for row in sample:
    print(f"\n'>>> Text: {row['text']}'")


'>>> Text: Or vus membre des granz amurs'

'>>> Text: '

'>>> Text: Ki sachent sun mal meciner:'

'>>> Text: '

'>>> Text: '

'>>> Text: '

'>>> Text: Tant qu’il furent al repeirer.'

'>>> Text: Mais n’irt mie bien a loer'

'>>> Text: "Veez le fol! Hu! hu! hu! hu!"'

'>>> Text: Al meins ore i unt tant perdu,'

'>>> Text: Des granz peines e des tristurs'

'>>> Text: '

'>>> Text: Baillez moi la, jo la prendrai:'

'>>> Text: '

'>>> Text: terre de Provence. Et Nicolete issi fors, si prist se viele, si'

'>>> Text: Ren ne desire se vus nun:'

'>>> Text: '

'>>> Text: '

'>>> Text: Ki nos deit lïer en amur.'

'>>> Text: '


In [26]:
chunk_size = 128

In [27]:
# Slicing produces a list of lists for each feature
tokenized_samples = tokenized_datasets["train"][:10]

for idx, sample in enumerate(tokenized_samples["input_ids"]):
    print(f"'>>> Text {idx} length: {len(sample)}'")

'>>> Text 0 length: 29'
'>>> Text 1 length: 29'
'>>> Text 2 length: 29'
'>>> Text 3 length: 29'
'>>> Text 4 length: 29'
'>>> Text 5 length: 29'
'>>> Text 6 length: 29'
'>>> Text 7 length: 29'
'>>> Text 8 length: 29'
'>>> Text 9 length: 29'


In [28]:
concatenated_examples = {
    k: sum(tokenized_samples[k], []) for k in tokenized_samples.keys()
}
total_length = len(concatenated_examples["input_ids"])
print(f"'>>> Concatenated text length: {total_length}'")

'>>> Concatenated text length: 290'


In [29]:
chunks = {
    k: [t[i : i + chunk_size] for i in range(0, total_length, chunk_size)]
    for k, t in concatenated_examples.items()
}

for chunk in chunks["input_ids"]:
    print(f"'>>> Chunk length: {len(chunk)}'")

'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 34'


In [30]:
def group_texts(examples):
    # Concatenate all texts
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    # Compute length of concatenated texts
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the last chunk if it's smaller than chunk_size
    total_length = (total_length // chunk_size) * chunk_size
    # Split by chunks of max_len
    result = {
        k: [t[i : i + chunk_size] for i in range(0, total_length, chunk_size)]
        for k, t in concatenated_examples.items()
    }
    # Create a new labels column
    result["labels"] = result["input_ids"].copy()
    return result

In [31]:
lm_datasets = tokenized_datasets.map(group_texts, batched=True)
lm_datasets

Loading cached processed dataset at /root/.cache/huggingface/datasets/parquet/bryantpwhite--Medieval_Texts_in_French-213d8ecd3f6cc3b0/0.0.0/1638526fd0e8d960534e2155dc54fdff8dce73851f21f031d2fb9c2cf757c121/cache-51ef09f64f3a4cf8.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/parquet/bryantpwhite--Medieval_Texts_in_French-213d8ecd3f6cc3b0/0.0.0/1638526fd0e8d960534e2155dc54fdff8dce73851f21f031d2fb9c2cf757c121/cache-c565bcff862e5ef1.arrow


DatasetDict({
    train: Dataset({
        features: ['attention_mask', 'input_ids', 'labels', 'word_ids'],
        num_rows: 2491
    })
    test: Dataset({
        features: ['attention_mask', 'input_ids', 'labels', 'word_ids'],
        num_rows: 2999
    })
})

In [32]:
tokenizer.decode(lm_datasets["train"][3]["input_ids"])

'<pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><s></s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><s></s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><s></s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><s></s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>'

In [33]:
train_size = 1000
test_size = int(0.1 * train_size)

downsampled_dataset = lm_datasets["train"].train_test_split(
    train_size=train_size, test_size=test_size, seed=42
)
downsampled_dataset

DatasetDict({
    train: Dataset({
        features: ['attention_mask', 'input_ids', 'labels', 'word_ids'],
        num_rows: 1000
    })
    test: Dataset({
        features: ['attention_mask', 'input_ids', 'labels', 'word_ids'],
        num_rows: 100
    })
})

In [34]:
from transformers import TrainingArguments

batch_size = 64
# Show the training loss with every epoch
logging_steps = len(downsampled_dataset["train"]) // batch_size
model_name = model_checkpoint.split("/")[-1]

training_args = TrainingArguments(
    output_dir=f"{model_name}-camembert-base",
    overwrite_output_dir=True,
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    push_to_hub=True,
    fp16=True,
    logging_steps=logging_steps,
)

In [47]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=downsampled_dataset["train"],
    eval_dataset=downsampled_dataset["test"],
    data_collator=data_collator,
    tokenizer=tokenizer
)

Cloning https://huggingface.co/bryantpwhite/camembert-base-camembert-base into local empty directory.
Using amp half precision backend


In [52]:
predictions = trainer.predict(tokenized_datasets["validation"])
print(predictions.predictions.shape, predictions.label_ids.shape)

KeyError: ignored

In [49]:
import numpy as np

preds = np.argmax(predictions.predictions, axis=-1)

NameError: ignored

In [48]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `CamembertForMaskedLM.forward` and have been ignored: word_ids.
***** Running training *****
  Num examples = 1000
  Num Epochs = 3
  Instantaneous batch size per device = 64
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 1
  Total optimization steps = 48


Epoch,Training Loss,Validation Loss
1,6.940600,5.725973
2,5.729000,5.289807
3,5.510400,5.406382


The following columns in the evaluation set  don't have a corresponding argument in `CamembertForMaskedLM.forward` and have been ignored: word_ids.
***** Running Evaluation *****
  Num examples = 100
  Batch size = 64
The following columns in the evaluation set  don't have a corresponding argument in `CamembertForMaskedLM.forward` and have been ignored: word_ids.
***** Running Evaluation *****
  Num examples = 100
  Batch size = 64
The following columns in the evaluation set  don't have a corresponding argument in `CamembertForMaskedLM.forward` and have been ignored: word_ids.
***** Running Evaluation *****
  Num examples = 100
  Batch size = 64


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=48, training_loss=6.0283792813618975, metrics={'train_runtime': 27.3895, 'train_samples_per_second': 109.531, 'train_steps_per_second': 1.752, 'total_flos': 197407026432000.0, 'train_loss': 6.0283792813618975, 'epoch': 3.0})

In [37]:
print(input_ids)


NameError: ignored

### Ashley's Troubleshooting Edits End

In [38]:
batch_sentences="text"

In [39]:
batch = tokenizer(batch_sentences, padding=True, truncation=True, return_tensors="tf")
print(batch)

{'input_ids': <tf.Tensor: shape=(1, 4), dtype=int32, numpy=array([[    5,    21, 12584,     6]], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(1, 4), dtype=int32, numpy=array([[1, 1, 1, 1]], dtype=int32)>}


In [40]:
tokenized_datasets=tokenized_datasets.map(lambda x: {'labels':x["input_ids"]})
print(tokenized_datasets)

  0%|          | 0/12140 [00:00<?, ?ex/s]

  0%|          | 0/18384 [00:00<?, ?ex/s]

DatasetDict({
    train: Dataset({
        features: ['attention_mask', 'input_ids', 'word_ids', 'labels'],
        num_rows: 12140
    })
    test: Dataset({
        features: ['attention_mask', 'input_ids', 'word_ids', 'labels'],
        num_rows: 18384
    })
})


In [41]:
tokenized_datasets

DatasetDict({
    test: Dataset({
        features: ['attention_mask', 'input_ids', 'word_ids', 'labels'],
        num_rows: 18384
    })
    train: Dataset({
        features: ['attention_mask', 'input_ids', 'word_ids', 'labels'],
        num_rows: 12140
    })
})

In [42]:
train_size = 10000
test_size = int(0.1 * train_size)

downsampled_dataset = tokenized_datasets["train"].train_test_split(
    train_size=train_size, test_size=test_size, seed=42
)
downsampled_dataset

DatasetDict({
    train: Dataset({
        features: ['attention_mask', 'input_ids', 'word_ids', 'labels'],
        num_rows: 10000
    })
    test: Dataset({
        features: ['attention_mask', 'input_ids', 'word_ids', 'labels'],
        num_rows: 1000
    })
})

In [41]:
!git config --global credential.helper store

In [42]:
from huggingface_hub import notebook_login

notebook_login()

Login successful
Your token has been saved to /root/.huggingface/token


In [43]:
from transformers import TrainingArguments

batch_size = 64
# Show the training loss with every epoch
logging_steps = len(downsampled_dataset["train"]) // batch_size
model_name = model_checkpoint.split("/")[-1]

training_args = TrainingArguments(
    output_dir=f"{model_name}-camembert-base",
    overwrite_output_dir=True,
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    push_to_hub=True,
    fp16=True,
    logging_steps=logging_steps,
)

In [44]:
#Instantiate data collator
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.15)

In [46]:
!apt install git-lfs 

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  git-lfs
0 upgraded, 1 newly installed, 0 to remove and 37 not upgraded.
Need to get 2,129 kB of archives.
After this operation, 7,662 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 git-lfs amd64 2.3.4-1 [2,129 kB]
Fetched 2,129 kB in 1s (2,466 kB/s)
Selecting previously unselected package git-lfs.
(Reading database ... 155229 files and directories currently installed.)
Preparing to unpack .../git-lfs_2.3.4-1_amd64.deb ...
Unpacking git-lfs (2.3.4-1) ...
Setting up git-lfs (2.3.4-1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...


In [39]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=downsampled_dataset["train"],
    eval_dataset=downsampled_dataset["test"],
    data_collator=data_collator,
    tokenizer=tokenizer
)

/content/gdrive/My Drive/Colab Notebooks/camembert-base-camembert-base is already a clone of https://huggingface.co/bryantpwhite/camembert-base-camembert-base. Make sure you pull the latest changes with `repo.git_pull()`.
Using amp half precision backend


In [40]:
from transformers.tokenization_utils_base import BatchEncoding

batch = BatchEncoding({"inputs": [[1, 2, 3], [4, 5, 6]], "labels": [0, 1]})
tensor_batch = batch.convert_to_tensors(tensor_type="tf",prepend_batch_axis=True)
print(tensor_batch)

{'inputs': <tf.Tensor: shape=(1, 2, 3), dtype=int32, numpy=
array([[[1, 2, 3],
        [4, 5, 6]]], dtype=int32)>, 'labels': <tf.Tensor: shape=(1, 2), dtype=int32, numpy=array([[0, 1]], dtype=int32)>}


In [41]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `CamembertForMaskedLM.forward` and have been ignored: word_ids.
***** Running training *****
  Num examples = 10000
  Num Epochs = 3
  Instantaneous batch size per device = 64
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 1
  Total optimization steps = 471


ValueError: ignored